## <font color="#59B4C3">🔨&nbsp;&nbsp;Google Colab Setup</font>
Upload the data (without unzipping it) to your Google Drive to be able to access datasets.



## <font color="#59B4C3">🎯&nbsp;&nbsp;Your task</font>

Take some time to play with this notebook (you will need to run the code this time) to gain a deeper understanding of the latent space. While there are no specific questions associated with this notebook, its content is important and it shouldn't take you too much time.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/dcgan.zip

In [ ]:
!pip install tf_keras

## <font color="#59B4C3">🧺&nbsp;&nbsp;Load Generator Model</font>

In [ ]:
import tf_keras as keras # We need to use tf_keras because the models saved were trained before Keras 3
generator = keras.models.load_model(f'dcgan/generator_1', compile=False)

## <font color="#59B4C3">🎩&nbsp;&nbsp;Generate Some Images</font>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

n_images = 100

# Remove seed if you want different images each time you run this cell
random_latent_vectors = tf.random.normal(shape=(n_images, 128), seed=1000)
generated_images = generator(random_latent_vectors)
generated_images = np.uint8(generated_images.numpy() * 127.5 + 127.5)

plt.figure(figsize=(12,12))

for i in range(n_images):
    plt.subplot(10, 10, i+1)
    plt.subplots_adjust(top=0.8, hspace=0.6)
    plt.title(str(i))
    plt.axis("off")
    plt.imshow(generated_images[i].squeeze(), cmap="gray")

plt.show()

## <font color="#59B4C3">🎨&nbsp;&nbsp;What Is Latent Space?</font>

The latent space is a multi-dimensional space the encode data into more abstract features.

Imagine you where to compare clothes images yourself, you would think about them with some abstractions (does it have sleeves? For which body part is it? ...) on some scales (sleeves lenght, ...) instead of comparing images pixels values one by one.

That's what we're doing with machine learning. We learn a meaningful way to encode the data instead of working directly on the whole data and then we can either do classification, regression, or reconstruct back data.

Most of the time the features it will learn will not be directly interpretable but some models architectures are designed in a way to have features we can understand (e.g. InfoGAN).

![Latent Space](https://miro.medium.com/max/720/0*kHJ_LsPi-jz_CreZ.webp)

In our case with DCGAN, we don't have the encoding part.

## <font color="#59B4C3">🔬&nbsp;&nbsp;Exploring GAN Latent Space</font>

Remember that with DCGAN, input vectors values are latent variables: features describing the object that are "hidden" because we are not directly able to measure or observe them.

We have as many dimensions in the latent space as we have values in the input vector. Thus a latent vector with two values form a space with two dimensions and one with 128 values form a space with 128 dimensions.

Each images we generate are sampled from this latent space, all of them have a feature vector representation that lies in this feature space.

When we understand that, we can as well understand that we can go from one generated image to another by moving from the first latent vector to the other one (we are moving from one point in the latent space to another one).

In [ ]:
def image_generated_from_latent_vector(latent_vector):
  generated_image = generator(np.expand_dims(latent_vector, axis=0))[0]

  print("Latent vector: ", latent_vector)

  plt.title("Image generated")
  plt.imshow(np.uint8(generated_image.numpy() * 127.5 + 127.5).squeeze(), cmap="gray")
  plt.show()

### Feature vector example

You can see below an example of a feature vector and the image we've generated from it.

In [ ]:
image_generated_from_latent_vector(random_latent_vectors[0])

### <font color="#59B4C3">🌍&nbsp;&nbsp;Moving in the latent space from images to images</font>

In [ ]:
def plot_linear_interpolation(vector_index1, vector_index2, vector_index3, n_step=10):
  move_vector_x = (random_latent_vectors[vector_index2] - random_latent_vectors[vector_index1]) / n_step
  move_vector_y = (random_latent_vectors[vector_index3] - random_latent_vectors[vector_index1]) / n_step

  fig, axs = plt.subplots(n_step, n_step, figsize=(n_step, n_step))

  axs[0, 0].set_title(vector_index1)
  axs[0, n_step - 1].set_title(vector_index2)
  axs[n_step - 1, 0].set_title(vector_index3, x=-0.25, y=0.3)

  for i in np.arange(n_step):
    for j in np.arange(n_step):
      generated_image = generator(np.expand_dims(random_latent_vectors[vector_index1] + move_vector_y * i + move_vector_x * j, axis=0))[0]
      axs[i,j].axis('off')
      axs[i,j].imshow(np.uint8(generated_image.numpy() * 127.5 + 127.5).squeeze(), cmap="gray")

  plt.show()

Choose the latent vector indexes and see how we can travel from one image to another. The indexes are the same as the number shown in the part where we show some images generated with the generator model.

The grid it will create below is like so: on the x axis we go from the first selected image (top left) to the second selected image (top right) by taking the difference between the two vectors and dividing this difference into _n_ number of steps. We're doing the same with the y axis, going from the first selected image (top left) to the third one (bottom left).

On the diagonal, we're moving in both the direction of the second image and the direction of the third one.

Click "Run Interact" to see the result.

In [ ]:
from ipywidgets import interact_manual
import ipywidgets as widgets

style = {'description_width': 'initial'}

interact_manual(
    plot_linear_interpolation,
    vector_index1=widgets.BoundedIntText(min=0, max=99, step=1, value=0, style=style),
    vector_index2=widgets.BoundedIntText(min=0, max=99, step=1, value=1, style=style),
    vector_index3=widgets.BoundedIntText(min=0, max=99, step=1, value=2, style=style),
    n_step=widgets.IntSlider(min=2, max=20, step=1, value=10)
)

See below an example of moving in the latent space of a stable diffusion model (similar to DALL-E):

>Latent space walking, or latent space exploration, is the process of sampling a point in latent space and incrementally changing the latent representation. Its most common application is generating animations where each sampled point is fed to the decoder and is stored as a frame in the final animation. For high-quality latent representations, this produces coherent-looking animations. These animations can provide insight into the feature map of the latent space, and can ultimately lead to improvements in the training process. One such GIF is displayed below:\
![Latent Space Walk](https://keras.io/img/examples/generative/random_walks_with_stable_diffusion/panda2plane.gif)\
>Source: https://keras.io/examples/generative/random_walks_with_stable_diffusion/

## <font color="#59B4C3">🔧&nbsp;&nbsp;Selecting Features Values Manually</font>

Here, you can see how it's not easy to understand the role of each features. Try to change the latent features values and to see the impact.

In [ ]:
from ipywidgets import Layout, HBox, VBox, interactive

def select_feature_vector(**kwargs):
  image_generated_from_latent_vector(list(kwargs.values()))

features_widget_list = {
    f"feature_{idx}": widgets.FloatSlider(min=-4, max=4, step=0.05, value=0)
    for idx in np.arange(128)
}

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='70%')

widget = interactive(
    select_feature_vector,
    {'manual': True},
    **features_widget_list,
)

controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
output.layout.height = '350px'

display(VBox([controls, output]))